In [1]:
# Check Python Version
!python --version

Python 3.10.12


In [2]:
# Check Ubuntu Version
!lsb_release -a

No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 22.04.2 LTS
Release:	22.04
Codename:	jammy


In [3]:
# Check CUDA/cuDNN Version
!nvcc -V && which nvcc

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0
/usr/local/cuda/bin/nvcc


In [4]:
# Check GPU
!nvidia-smi

Sun Aug 20 21:25:23 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
# This get the RAPIDS-Colab install files and test check your GPU.  Run this and the next cell only.
# Please read the output of this cell.  If your Colab Instance is not RAPIDS compatible, it will warn you and give you remediation steps.
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 390, done.
remote: Counting objects: 100% (121/121), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 390 (delta 89), reused 51 (delta 51), pack-reused 269
Receiving objects: 100% (390/390), 107.11 KiB | 5.95 MiB/s, done.
Resolving deltas: 100% (191/191), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 2.0 MB/s eta 0:00:00
***********************************************************************
Woo! Your instance has the right kind of GPU, a Tesla T4!
We will now install RAPIDS cuDF, cuML, and cuGraph via pip! 
Please stand by, should be quick...
***********************************************************************

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.6/499.6 MB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 GB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [15]:
import cudf
import pandas as pd

# Load the CSV file into a cuDF DataFrame
cudf_df = cudf.read_csv('cluster_cpu_util_full.csv')

# Convert the 'time' column to datetime type using cuDF
cudf_df['time'] = cudf.to_datetime(cudf_df['time'])

# Convert cuDF DataFrame to pandas DataFrame for resampling
pandas_df = cudf_df.to_pandas()

# Group by 'machine_id' and resample the data to hourly intervals using pandas
resampled_df = pandas_df.groupby(['machine_id', pd.Grouper(key='time', freq='60T')])['cpu_util'].mean().reset_index()

# Convert the resampled pandas DataFrame back to cuDF
resampled_cudf_df = cudf.DataFrame.from_pandas(resampled_df)

# Save the resampled cuDF DataFrame to a new CSV file
resampled_cudf_df.to_pandas().to_csv('resampled_cluster_cpu_util.csv', index=False)

print("Resampling and averaging done. Saved to 'resampled_cluster_cpu_util.csv'.")


/usr/local/lib/python3.10/dist-packages/dateutil/parser/_parser.py:1207: UnknownTimezoneWarning: tzname PST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


Resampling and averaging done. Saved to 'resampled_cluster_cpu_util.csv'.


In [16]:
import pandas as pd

df = pd.read_csv('resampled_cluster_cpu_util.csv')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18626 entries, 0 to 18625
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   machine_id  18626 non-null  object 
 1   time        18626 non-null  object 
 2   cpu_util    18456 non-null  float64
dtypes: float64(1), object(2)
memory usage: 436.7+ KB
